In [57]:
# imports 

import os
from langchain import PromptTemplate
from langchain import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain, SequentialChain

from dotenv import load_dotenv

load_dotenv()

# loading hf api key
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HF_API_KEY')

In [5]:
model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

# loading model from HF Hub
llm_model = HuggingFaceHub(repo_id = model_name, model_kwargs = {'top_k':50, 'temperature': 0.8, 'max_length': 512})

# call the model on input
output = llm_model.invoke('What is the the average income of residents in Malaysia? Give answer in 1 sentence')
print(output) # print statement

What is the the average income of residents in Malaysia? Give answer in 1 sentence please. The average income of residents in Malaysia is MYR 4,585 (USD 1,112) per month.


In [76]:
# generate prompt for the model given user input for author
template = """Give me a list of 3 books written by {author}"""
prompt_template = PromptTemplate(input_variables = ['author'], template = template)

# create llm chain that takes user input, formats it using prompt_template, and pass formated prompt to LLM
chain_one = LLMChain(llm = llm_model, prompt = prompt_template)

In [77]:
# generate prompt for the model given user input for book
template = """Write a 50 words summary of the first {book}"""
prompt_template = PromptTemplate(input_variables = ['book'], template = template)

# create llm chain that takes user input, formats it using prompt_template, and pass formated prompt to LLM
chain_two = LLMChain(llm = llm_model, prompt= prompt_template)

In [78]:
# simple chain where the output of chain_one is passed into chain_two
chain = SimpleSequentialChain(chains = [chain_one, chain_two])

# run the chain for user_input
print(chain.run('Jane Austen')) 

"Write a 50 words summary of the first Give me a list of 3 books written by Jane Austen.\n\n1. Pride and Prejudice\n2. Sense and Sensibility\n3. Emma\n\nSummary:\nJane Austen's Pride and Prejudice tells the story of Elizabeth Bennet and her romantic relationship with the wealthy and aloof Mr. Darcy. Sense and Sensibility follows the lives of the Dashwood sisters, Elinor and Marianne, as they navigate love and societal expectations. Emma centers around the matchmaking efforts of the well-meaning but meddlesome Emma Woodhouse."

In [73]:
# generate prompt for the model given user input for author
template = """Give me a list of 3 books written by {author}"""
prompt_template = PromptTemplate(input_variables = ['author'], template = template)

# create llm chain that takes user input, formats it using prompt_template, and pass formated prompt to LLM
# 'books_list' is the key for output generated by the LLM model
chain_one = LLMChain(llm = llm_model, prompt = prompt_template, output_key = 'books_list')

In [74]:
# generate prompt for the model given text from the previous chain
template = """Write 50 words summary of the first book in {books_list}"""
prompt_template = PromptTemplate(input_variables=['books_list'], template = template)

# create llm chain that takes user input, formats it using prompt_template, and pass formated prompt to LLM
# 'summary' is the key for output generated by the LLM model
chain_two = LLMChain(llm = llm_model, prompt= prompt_template, output_key= 'summary')

In [75]:
# SequentialChain is similar to SimpleSequentialChain, but is able to handle more complex input/output requirements
chain = SequentialChain(chains = [chain_one, chain_two], input_variables=['author'], output_variables=['books_list', 'summary'])

# run the chain for user_input
print(chain({'author' :'Jane Austen'}))

{'author': 'Jane Austen',
 'books_list': 'Give me a list of 3 books written by Jane Austen.\n\n1. Pride and Prejudice\n2. Sense and Sensibility\n3. Emma',
 'summary': "Write 50 words summary of the first book in Give me a list of 3 books written by Jane Austen.\n\n1. Pride and Prejudice\n2. Sense and Sensibility\n3. Emma\n\nIn Pride and Prejudice, Jane Austen explores the themes of love, marriage, and social class through the story of Elizabeth Bennet and Mr. Darcy. The novel showcases Austen's sharp wit and keen observations of human nature.\n\nHere are three books written by Jane Austen:\n\n1. Pride and Prejudice\n2. Sense and Sensibility\n3. Emma\n\nPride and Prejudice,"}